# DNA Sequences

In this notebook, I explore the `Seq` class for representing DNA sequences.

In [1]:
%load_ext autoreload
%autoreload 2

import set_dir

In [2]:
from Sequence import Seq

## Basic Functionality

Defining a sequence:

In [3]:
s1 = Seq('ACGT')
print(s1)
print(repr(s1))
print(len(s1))

ACGT
<ACGT>
4


Comparisons:

In [4]:
s2 = Seq('AAAC')
print(s1 == s2)
print(s1 != s2)
print(s1  > s2)

False
True
True


Concatenating sequences:

In [5]:
# Concat two sequences
print("Concatenation")
s1 = Seq("AAA")
s2 = Seq("GGG")
s3 = s1 + s2
print(f"{s1} + {s2} = {s3}")

# Concat sequence with string
print(s1 + "CCC")
print("CCC" + s1)

# Concat multiple sequences
s4 = Seq("TTT")
print(Seq.concat(s1, s2, s4))
print(Seq.concat( ( s for s in [s1, s2, "CCC", s4] ) ))
print("")

Concatenation
AAA + GGG = AAAGGG
AAACCC
CCCAAA
AAAGGGTTT
AAAGGGCCCTTT



Repeating a sequence:

In [6]:
print("Repetition")
print(Seq("AC") * 5)
print("")

Repetition
ACACACACAC



Searching for subsequences:

In [7]:
# Checking for subsequence
print("Subsequence")
print(Seq("AA") in Seq("CAAC"))
print(Seq("TG") in Seq("CAAC"))
print("AA" in Seq("AAAA"))
print("")

Subsequence
True
False
True



Complements:

In [8]:
print(s1)
print(~s1)

# Checking modulus (sequence is otw own complement)
print("Modulus:")
print(Seq("ACGT") % Seq("ACGT"))
print("")

print(s1 % s1)
print(s4 % s4)
print(s4 % "AGCT")

AAA
TTT
Modulus:
True

False
False
False


## Sequence Matching

I have defined the logical operators `&`, `|`, and `^`, as well as `-`, to perform pairwise operations on two sequences.

Much of the interesting nature of these operators comes from the extended use of the letters in the FASTA format, i.e. the letters which represent that more than one base is possible.

- `and` returns the bases if they match, and a gap if they don't.  For example, `ACA & ATR` will be `A-A`.  The first pair matches, so they are kept; the second pair does not, so a gap is inserted; and in the third pair, `R` can represent `A` or `T/U`, so only `A` is kept.
- `or` returns the most restrictive code that could represent both sequences.  For example, `A | T` will be `R`, since `R` can represent either `A` or `T/U`.
- `xor` is similar to `and`.  If the bases match, they are kept, as in `and`; if not, a gap is inserted.  For example, `ACA ^ ATR` will be `A--`, since `A` and `R` are not equal.
- `sub` removes the ability for the first sequence to represent the second.  For example, if `seq1` is `N` and `seq2` is `A`, then `seq1 - seq2` will be `B`, which can represent `C`, `G`, or `T/U`, but not `A`.

If a sequence does not already contain extended bases, `or` is the only operation that will introduce them.

In [9]:
s1 = Seq("AAG-TA")
s2 = Seq("AGGC-R")

print(s1 & s2)
print(s1 ^ s2)
print(s1 | s2)
print("")

print( s1 >> 3 )


s1 = Seq("AAAAAAA")
s2 = Seq("ACGTRB-")
print(s1 - s2)

print( s1 & "ACACACA" )
print( "ACACACA" & s1 )
s1 &= "ACACACA"
print(s1)

print ("GATTACA" - s1)

A-G--A
A-GCT-
ARGCTR

---AAG-TA
-AAA-AA
A-A-A-A
A-A-A-A
A-A-A-A
GATT-C-


## Transforming Sequences

### RNA

In [10]:
from Sequence import Seq

print(f"Class method, Seq input:   {Seq.dna_to_rna(Seq('ACGT'))}")
print(f"Class method, Str input:   {Seq.dna_to_rna(    'ACGT' )}")
print(f"Object method:             {Seq('ACGT').to_RNA()}"       )

Class method, Seq input:   ACGU
Class method, Str input:   ACGU
Object method:             ACGU


### Amino Acids

In [11]:
from Sequence import Seq

s1 = Seq("AAAGGG")
    
print(Seq.translate(s1))
print(s1.translate())

KG
KG


# Benchmarking

In the following sections, I benchmark how the array implementation and string implementation compare to one another on a few basic operations.

In the cell below, I define a generalized function to use the `timeit` module to time a simple function.

In [38]:
from random import randint
import timeit

def time_seq_func(init_func, test_func, num_trials=10000, length=6, closure=False):

    # Print the header
    print("| Seq Len | Total Time (s) |  Avg Time (s)  |")
    print("|---------|----------------|----------------|")

    # Test increasing sequence lengths
    for n in [ 10 ** n for n in range(1, length)]:

        # Create a random sequence of length 10^n
        seq = init_func()
        for i in range(n):
            seq += d[randint(0,3)]
        
        if closure:
            f = test_func(seq)
        else:
            f = test_func("seq")

        # Create the function to test by providing the local variable
        # name of the sequence, then run the tests
        t = timeit.timeit(
            f, number=num_trials, globals=locals()
        )

        # Print results
        print(f"| {n:>7} | {t:.6f} | {t/num_trials:.6f} |")

## Complement

Here, I time how long it takes to take the complement of random sequences of increasing length, for the array implementation and the string implementation.

In [47]:
from Sequence import Seq
from Sequence._sequenceStr import SeqStr

time_seq_func(Seq,    lambda name: f"~{name}", length=5)
time_seq_func(SeqStr, lambda name: f"~{name}", length=5)

| Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.092946 | 0.000009 |
|     100 | 0.379364 | 0.000038 |
|    1000 | 3.672137 | 0.000367 |
|   10000 | 30.971811 | 0.003097 |
| Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.013738 | 0.000001 |
|     100 | 0.128964 | 0.000013 |
|    1000 | 0.936815 | 0.000094 |
|   10000 | 9.050150 | 0.000905 |


## Repetition

In the following cells, I test the repetition operator `*`.

In [39]:
time_seq_func(Seq,    lambda name: f"{name} * 10", length=5)

| Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.032111 | 0.000003 |
|     100 | 0.038990 | 0.000004 |
|    1000 | 0.103301 | 0.000010 |
|   10000 | 0.720746 | 0.000072 |


In [40]:
time_seq_func(Seq,    lambda name: f"{name} * 10", length=5)

| Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.086479 | 0.000009 |
|     100 | 0.587518 | 0.000059 |
|    1000 | 5.564199 | 0.000556 |
|   10000 | 53.978736 | 0.005398 |


In [45]:
from genetics.sequenceStr import SeqStr
time_seq_func(SeqStr, lambda name: f"{name} * 10", length=5)

| Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.007559 | 0.000001 |
|     100 | 0.005302 | 0.000001 |
|    1000 | 0.010423 | 0.000001 |
|   10000 | 0.043862 | 0.000004 |


Timing the repeat operator `*`.

The left two columns show a native array implementation.

The middle two show an array implementation that converts to a `str` to use the native string implementation of the operator.

The right two show the string implementation.

| Seq Len | Total Time |  Avg Time  | Total Time | Avg Time | Total Time | Avg Time |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: |
|     10 | 0.032111 | 0.000003 |  0.086479 | 0.000009 | 0.007559 | 0.000001 |
|    100 | 0.038990 | 0.000004 |  0.587518 | 0.000059 | 0.005302 | 0.000001 |
|   1000 | 0.103301 | 0.000010 |  5.564199 | 0.000556 | 0.010423 | 0.000001 |
|  10000 | 0.720746 | 0.000072 | 53.978736 | 0.005398 | 0.043862 | 0.000004 |

All times are given in seconds.

In this case, converting the array to a string has by far the worst performance.  The array implementation is fast, but nowhere near the string method.

## Concatenation

### Basic Concatenation

For concatenation, I first test the basic `+` operator for a sequence and itself.

In [113]:
time_seq_func(Seq,    lambda name: f"{name} + {name}", length=6)
time_seq_func(SeqStr, lambda name: f"{name} + {name}", length=6)

| Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.014539 | 0.000001 |
|     100 | 0.015517 | 0.000002 |
|    1000 | 0.019539 | 0.000002 |
|   10000 | 0.040925 | 0.000004 |
|  100000 | 0.201383 | 0.000020 |
| Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.008414 | 0.000001 |
|     100 | 0.008804 | 0.000001 |
|    1000 | 0.010805 | 0.000001 |
|   10000 | 0.017672 | 0.000002 |
|  100000 | 0.123884 | 0.000012 |


Based on this data, the array method appears to take about twice as much time to compute.

### In-Place Concatenation

Here, I test the in-place concatenation operator `+=`, which modifies an object directly.

In [118]:
# Test increasing sequence lengths
for n in [ 10 ** n for n in range(1, 6)]:

    # Create a random sequence of length 10^n
    s = time()
    seq = Seq()
    for i in range(n):
        seq += d[randint(0,3)]
    t = time()

    # Print results
    print(f"| {n:>7} | {t-s:.6f} | {t-s:.6f} |")

# Test increasing sequence lengths
for n in [ 10 ** n for n in range(1, 6)]:

    # Create a random sequence of length 10^n
    s = time()
    seq = SeqStr()
    for i in range(n):
        seq += d[randint(0,3)]
    t = time()

    # Print results
    print(f"| {n:>7} | {t-s:.6f} | {t-s:.6f} |")

|      10 | 0.000997 | 0.000997 |
|     100 | 0.000000 | 0.000000 |
|    1000 | 0.000997 | 0.000997 |
|   10000 | 0.010981 | 0.010981 |
|  100000 | 0.119287 | 0.119287 |
|      10 | 0.000000 | 0.000000 |
|     100 | 0.000000 | 0.000000 |
|    1000 | 0.001994 | 0.001994 |
|   10000 | 0.022235 | 0.022235 |
|  100000 | 0.286513 | 0.286513 |


In this case, the array method is twice as fast as the string method.

<!-- | Seq Len | Total Time (s) |  Avg Time (s)  |
|---------|----------------|----------------|
|      10 | 0.021190 | 0.000002 |
|     100 | 0.149093 | 0.000015 |
|    1000 | 1.888353 | 0.000189 |
|   10000 | 15.995845 | 0.001600 |
|  100000 | 145.837629 | 0.014584 | -->

Left column is the array implementation, right column is the string implementation.

| Seq Len | Total Time (s) | Total Time (s) |
| --: | :-: | :-: |
|      10 | 0.000997 | 0.000000 |
|     100 | 0.000000 | 0.000000 |
|    1000 | 0.000997 | 0.001994 |
|   10000 | 0.010981 | 0.022235 |
|  100000 | 0.119287 | 0.286513 |

## Writing to File

Here, I test how long it takes to write both to a file.

In [119]:
from genetics.sequence import Seq

# from random import
from time import time

with open("output/test_file.txt", "w") as f:
    s = time()
    for i in range(10000):
        s1 = Seq("AAAA")
        s2 = Seq("CCCC")

        f.write(str(s1 + s2))
        f.write("\n")

t = time()
print(t - s)

with open("output/test_file.txt", "w") as f:
    s = time()
    for i in range(10000):
        s1 = "AAAA"
        s2 = "CCCC"

        f.write(s1 + s2)
        f.write("\n")

t = time()
print(t - s)

0.052541255950927734
0.014759302139282227


The string method is significantly faster. I suspect this is because we have to convert from the byte representation in the array to a string in order to save; it might be possible to write a dedicated `write` function for the array implementation that operates faster.

## Random Slicing

Here, I test how long it takes to access a random slice of the sequence.  I randomly generate two indices, order them, and write that slice to a file.

In [76]:
from time import time

# Test increasing sequence lengths
for n in [ 10 ** n for n in range(1, 6)]:

    # Create a random sequence of length 10^n
    seq = Seq()
    for i in range(n):
        seq += d[randint(0,3)]

    with open("output/test_file.txt", "w") as f:
        s = time()
        for it in range(10000):
            rand_1 = randint(0, len(seq))
            rand_2 = randint(0, len(seq))
            i, j = min(rand_1, rand_2), max(rand_1, rand_2)
            f.write(str(seq[i:j]))
            f.write("\n")
    t = time()

    # Print results
    print(f"| {n:>7} | {t-s:.6f} | {t-s:.6f} |")

# Test increasing sequence lengths
for n in [ 10 ** n for n in range(1, 6)]:

    # Create a random sequence of length 10^n
    seq = SeqStr()
    for i in range(n):
        seq += d[randint(0,3)]

    with open("output/test_file.txt", "w") as f:
        s = time()
        for it in range(10000):
            rand_1 = randint(0, len(seq))
            rand_2 = randint(0, len(seq))
            i, j = min(rand_1, rand_2), max(rand_1, rand_2)
            f.write(str(seq[i:j]))
            f.write("\n")
    t = time()

    # Print results
    print(f"| {n:>7} | {t-s:.6f} | {t-s:.6f} |")

|      10 | 0.045677 | 0.045677 |
|     100 | 0.076190 | 0.076190 |
|    1000 | 0.118940 | 0.118940 |
|   10000 | 0.429558 | 0.429558 |
|  100000 | 3.187925 | 3.187925 |
|      10 | 0.035199 | 0.035199 |
|     100 | 0.057343 | 0.057343 |
|    1000 | 0.086189 | 0.086189 |
|   10000 | 0.399790 | 0.399790 |
|  100000 | 2.983008 | 2.983008 |


Here, the two methods are comparable, with the array method only slightly slower than the string method (within ~0.2 seconds for 10,000 accesses of a sequence of length 100,000).